<a href="https://colab.research.google.com/github/javohir-ergashboyev/tensorflow_basics/blob/main/Save_and_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Saving and Loading weights at any time**

In [1]:
!pip install pyyaml h5py

In [3]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [5]:
data=mnist.load_data()

In [6]:
len(data)

2

In [14]:
len(data[0][1]),len(data[1][1])

(60000, 10000)

In [15]:
(xtrain,ytrain),(xtest,ytest)=data

In [16]:
len(ytrain)

60000

In [17]:
ytrain=ytrain[:1000]
ytest=ytest[:1000]

xtrain=xtrain[:1000].reshape(-1,784)/255.0
xtest=xtest[:1000].reshape(-1,784)/255.0

In [21]:
xtrain[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [19]:
def create_model():
  model=tf.keras.Sequential([
      layers.Dense(512, activation='relu', input_shape=(784,)),
      layers.Dropout(0.2),
      layers.Dense(10)
  ])

  model.compile(
      optimizer='adam',
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
  )
  return model
model=create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               401920    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**one checkpoint file**

In [22]:
checkpoint_path='training_1/cp.ckpt'
checkpoint_dir=os.path.dirname(checkpoint_path)
def get_callback():
  return [
      tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)
  ]


In [23]:
model.fit(
    xtrain,
    ytrain,
    epochs=10,
    callbacks=get_callback(),
    validation_data=(xtest,ytest)
)

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 1.1507 - sparse_categorical_accuracy: 0.6770
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 4s 15ms/step - loss: 1.1507 - sparse_categorical_accuracy: 0.6770 - val_loss: 0.7508 - val_sparse_categorical_accuracy: 0.7620
Epoch 2/10
20/32 [=================>............] - ETA: 0s - loss: 0.3993 - sparse_categorical_accuracy: 0.8844
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4102 - sparse_categorical_accuracy: 0.8790 - val_loss: 0.5141 - val_sparse_categorical_accuracy: 0.8330
Epoch 3/10
19/32 [================>.............] - ETA: 0s - loss: 0.2888 - sparse_categorical_accuracy: 0.9161
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.2667 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.4596 - val_sparse_categorical_accuracy: 0.8560
Epoch 4/10
20/32 [=========

In [24]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [25]:
test_model=create_model()
loss,acc=test_model.evaluate(xtest,ytest,verbose=2)
print('test model accuracy {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 2.3545 - sparse_categorical_accuracy: 0.1070 - 167ms/epoch - 5ms/step
test model accuracy 10.70%


In [26]:
test_model.load_weights(checkpoint_path)
loss,acc=test_model.evaluate(xtest,ytest,verbose=2)
print('test model accuracy {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.3961 - sparse_categorical_accuracy: 0.8680 - 82ms/epoch - 3ms/step
test model accuracy 86.80%


**Multiple checkpoint files**

In [29]:
checkpoints_path='training_2/cp-{epoch:04d}.ckpt'
checkpoints_dir=os.path.dirname(checkpoints_path)

import math

batch_size=32
n_batches=len(xtrain)/batch_size
n_batches=math.ceil(n_batches)

def get_callback2():
  return [
      tf.keras.callbacks.ModelCheckpoint(
          checkpoints_path,
          verbose=1,
          save_weights_only=True,
          save_freq=5*n_batches
      )
  ]

model=create_model()

model.save_weights(checkpoints_path.format(epoch=0))
model.fit(
    xtrain,
    ytrain,
    validation_data=(xtest,ytest),
    epochs=50,
    callbacks=get_callback2()
)



Epoch 1/50
32/32 [==============================] - 1s 10ms/step - loss: 1.2003 - sparse_categorical_accuracy: 0.6390 - val_loss: 0.7582 - val_sparse_categorical_accuracy: 0.7680
Epoch 2/50
32/32 [==============================] - 0s 5ms/step - loss: 0.4606 - sparse_categorical_accuracy: 0.8620 - val_loss: 0.5538 - val_sparse_categorical_accuracy: 0.8270
Epoch 3/50
32/32 [==============================] - 0s 5ms/step - loss: 0.2867 - sparse_categorical_accuracy: 0.9210 - val_loss: 0.4834 - val_sparse_categorical_accuracy: 0.8470
Epoch 4/50
32/32 [==============================] - 0s 5ms/step - loss: 0.2135 - sparse_categorical_accuracy: 0.9430 - val_loss: 0.4491 - val_sparse_categorical_accuracy: 0.8540
Epoch 5/50
20/32 [=================>............] - ETA: 0s - loss: 0.1719 - sparse_categorical_accuracy: 0.9641
Epoch 5: saving model to training_2/cp-0005.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.1664 - sparse_categorical_accuracy: 0.9660 - val_loss: 0.4215 

In [30]:
os.listdir(checkpoints_dir)

['cp-0040.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0025.ckpt.index',
 'checkpoint',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0020.ckpt.index']

In [31]:
latest=tf.train.latest_checkpoint(checkpoints_dir)
latest


'training_2/cp-0050.ckpt'

In [32]:
test_model=create_model()
test_model.load_weights(latest)
loss,acc=test_model.evaluate(xtest,ytest,verbose=2)
print('test model accuracy {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4761 - sparse_categorical_accuracy: 0.8760 - 158ms/epoch - 5ms/step
test model accuracy 87.60%
